# BERT Q/A Model for document search

In [88]:
import os
import glob
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
from tqdm import tqdm
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 100)

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

In [2]:
def absoluteFilePaths(directory):
    path = []
    files = []
    for dirpath, dirname, filenames in os.walk(directory):
        for f in filenames:
            if not os.path.basename(dirpath).startswith('.'):
                path.append(dirpath)
                files.append(f)
            
    return path, files

In [3]:
#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-cased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-cased-whole-word-masking-finetuned-squad')

In [4]:
DIRECTORY = os.getcwd()
locations, documents = absoluteFilePaths(os.path.join(DIRECTORY, 'Google', 'research'))
paths = [os.path.join(loc, doc) for loc, doc in zip(locations, documents)]

In [5]:
paths

['/home/jupyter/nlp_document_finder/Google/research/nlp.pdf',
 '/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf',
 '/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf']

In [6]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [7]:
contents = []
for path in paths:
    if path.endswith('.pdf'):
        contents.append(convert_pdf_to_txt(path))
    else: 
        continue

In [8]:
paths

['/home/jupyter/nlp_document_finder/Google/research/nlp.pdf',
 '/home/jupyter/nlp_document_finder/Google/research/computer_vision.pdf',
 '/home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf']

In [180]:
question = "What is attention mechanism"
answers = []
for path, content in tqdm(zip(paths, contents)):
    for i in range(0, len(content.split(" ")), 50):
        paragraph = content.split(" ")[i:i+50]
        encoding = tokenizer.encode_plus(text=question, text_pair=paragraph)
        inputs = encoding['input_ids']  #Token embeddings
        sentence_embedding = encoding['token_type_ids']  #Segment embeddings
        tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens
        
        start_scores, end_scores = model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))

        start_index = torch.argmax(start_scores)
        end_index = torch.argmax(end_scores)
        answer = ' '.join(tokens[start_index:end_index+1])
        if start_index.numpy() < end_index.numpy():
            answers.append([path, answer, i, i+start_index.numpy(), i+end_index.numpy(), (torch.max(start_scores)+torch.max(end_scores)).detach().numpy()])    

3it [03:37, 72.42s/it]


In [185]:
df_answers = pd.DataFrame(data=answers, columns = ['path', 'answer', 'chunk', 'start_loc', 'end_loc', 'logit'])
df_answers = df_answers.sort_values(by=['logit'], ascending=False).reset_index(drop=True)

In [186]:
df_answers.head()

path  \
0  /home/jupyter/nlp_document_finder/Google/research/quantum_computing.pdf   
1                /home/jupyter/nlp_document_finder/Google/research/nlp.pdf   
2                /home/jupyter/nlp_document_finder/Google/research/nlp.pdf   
3                /home/jupyter/nlp_document_finder/Google/research/nlp.pdf   
4                /home/jupyter/nlp_document_finder/Google/research/nlp.pdf   

                                                                                      answer  \
0                                                                                    The tip   
1                     compatibility function using a [UNK] network [UNK] single hidden [UNK]   
2  allows the model to jointly attend to information from different [UNK] at different [UNK]   
3                                                                              Two attention   
4                       the attention function on a set of [UNK] [UNK] packed [UNK] a matrix   

   chunk  start_loc  end_loc       logit  
0    950       1001     1002   10.519569  
1   1300       1328     1337   10.337739  
2   1500       1541     1554  10.0983715  
3   4950       4998     4999    9.729937  
4   1250       1258     1270    8.547232

In [192]:
" ".join(contents[0].split(" ")[1500: 1500+50])

'large, assume that the components of q and k are independent random\ni=1 qiki, has mean 0 and variance dk.\n\nvariables with mean 0 and variance 1. Then their dot product, q · k = (cid:80)dk\n\n4\n\n\x0cMulti-head attention allows the model to jointly attend to information from different representation\nsubspaces at different positions. With'